<a href="https://colab.research.google.com/github/IngaKristin/Final-Project-TensorFlow/blob/vera-branch/dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important Links:**
- [Paper](https://arxiv.org/pdf/2011.13062.pdf)
- [GitHub](https://github.com/sfc-computational-creativity-lab/x-rhythm-can)

1. [Donwload](https://magenta.tensorflow.org/datasets/groove#format) the Groove MIDI Dataset
2. Mount the dataset to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. Load the Dataset**

##### **1.1 Before loading the dataset, let's have a look at the `info.csv` file.**

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('/content/drive/MyDrive/TF-dataset/groove/info.csv')
df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


##### **1.2 Next, let's have a look at the column names and their meaning.**

In [37]:
df.columns

Index(['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split'],
      dtype='object')

`drummer` <br>
`session` <br>
`id` <br>
`style` <br>
`bpm` <br>
`beat_type` <br>
`time_signature` <br>
`midi_filename` <br>
`audio_filename` <br>
`duration` <br>
`split` <br>

In [33]:
df.info

<bound method DataFrame.info of        drummer                session                        id  \
0     drummer1  drummer1/eval_session   drummer1/eval_session/1   
1     drummer1  drummer1/eval_session  drummer1/eval_session/10   
2     drummer1  drummer1/eval_session   drummer1/eval_session/2   
3     drummer1  drummer1/eval_session   drummer1/eval_session/3   
4     drummer1  drummer1/eval_session   drummer1/eval_session/4   
...        ...                    ...                       ...   
1145  drummer2      drummer2/session2      drummer2/session2/11   
1146  drummer2      drummer2/session2      drummer2/session2/12   
1147  drummer2      drummer2/session2      drummer2/session2/13   
1148  drummer2      drummer2/session2      drummer2/session2/14   
1149  drummer2      drummer2/session2      drummer2/session2/15   

              style  bpm beat_type time_signature  \
0      funk/groove1  138      beat            4-4   
1     soul/groove10  102      beat            4-4   
2   

In [34]:
count_train = df[df['split'] == 'train'].count()
count_test = df[df['split'] == 'test'].count()
count_train
#f"The number of train data is {count_train}, {count_test}"

drummer           897
session           897
id                897
style             897
bpm               897
beat_type         897
time_signature    897
midi_filename     897
audio_filename    846
duration          897
split             897
dtype: int64

In [1]:
import tensorflow as tfds